### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
            
    #l_class_count, l_sizes, r_class_count, r_sizes
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (np.sum((l_c / l_s)**2, axis=1) + np.sum((r_c / r_s)**2, axis=1))
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum((l_c / l_s) * np.log2(l_c / l_s), axis=1) + np.sum((r_c / r_s) * np.log2(r_c / r_s), axis=1))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max((l_c / l_s), axis=1) + np.max((r_c / r_s), axis=1))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    #возвращает наименьшее значение меры неопределенности и 
    #значение порога критерия для успешного разделения в узле на 2 части
    def __find_threshold(self, x, y):
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (cut_size + 1)
        if len(r_border_ids) == 0:
            return np.inf, None
        eq_el_count = r_border_ids - np.append(np.array([cut_size]), r_border_ids[:-1])
        #1 - класс поменялся, 0 - не поменялся
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut_size], minlength=class_number)
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y, minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        return gs[idx], (sorted_x[l_sizes[idx][0] - 1] + sorted_x[l_sizes[idx][0]]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        n_samples = x.shape[0]
        y = y.astype(int)
        num_cl = np.bincount(y)
        maxnum_cl = num_cl.argmax()

        if self.max_depth == depth or n_samples <= self.min_samples_split or\
           self.sufficient_share <= float(maxnum_cl)/n_samples:
            self.tree[node_id] = (self.LEAF_TYPE, maxnum_cl)
            return
        else:
            n_features = x.shape[1]
            gs = np.zeros(n_features).astype(float)
            thresholds = np.zeros(n_features).astype(float)
            for i in self.get_feature_ids(n_features):
                gs[i], thresholds[i] = self.__find_threshold(x[:, i], y)
            f_id_best = np.argmin(gs)
            threshold = thresholds[f_id_best]

            if np.all(self.feature_importances_ is None):
                self.feature_importances_ = np.zeros(x.shape[1], dtype=float)
            self.feature_importances_[f_id_best] = gs[f_id_best]/n_samples

            x_l, x_r, y_l, y_r = self.__div_samples(x, y, f_id_best, threshold)

            if (x_l.shape[0] == 0 or x_r.shape[0] == 0):
                self.tree[node_id] = (self.LEAF_TYPE, maxnum_cl)
                return

            self.tree[node_id] = (self.NON_LEAF_TYPE, f_id_best, threshold)
            self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier()
clf = DecisionTreeClassifier()

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [33]:
%time clf.fit(X_train, y_train)

CPU times: user 419 µs, sys: 3.41 ms, total: 3.83 ms
Wall time: 5.57 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [34]:
%time my_clf.fit(X_train, y_train)

CPU times: user 60.5 ms, sys: 0 ns, total: 60.5 ms
Wall time: 59.7 ms


## Проверка качества работы на wine

In [35]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [36]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8968253968253969

## Подготовка данных Speed Dating Data 

In [37]:
df = pd.read_csv('data/speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251')
print(df.shape)
df.head()

(8378, 195)


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [38]:
df = df.drop(['id','idg','condtn','round','position', 'positin1','order','partner','age_o','race_o','pf_o_att',
              'pf_o_sin','pf_o_int','pf_o_fun','pf_o_amb','pf_o_sha','dec_o','attr_o','sinc_o','intel_o',
              'fun_o','amb_o','shar_o','like_o','prob_o','met_o','field','undergra','mn_sat','tuition','from',
              'zipcode','career','sports','tvsports','exercise','dining','museums','art','hiking','gaming',
              'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga',
              'attr1_1','sinc1_1','intel1_1','fun1_1','amb1_1','shar1_1',
              'attr4_1','sinc4_1','intel4_1','fun4_1','amb4_1','shar4_1',
              'attr2_1','sinc2_1','intel2_1','fun2_1','amb2_1','shar2_1',
              'attr3_1','sinc3_1','intel3_1','fun3_1','amb3_1',
              'attr5_1','sinc5_1','intel5_1','fun5_1','amb5_1',
              'attr','sinc','intel','fun','amb','shar','like','prob','met','match_es','dec','wave'], axis=1)

In [39]:
df = df.dropna(subset=['pid','int_corr','age'])#df.age.isnull().sum()

In [40]:
np.where(df.field_cd.isnull())[0]

array([488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500,
       501, 502, 503, 504, 505])

In [41]:
df.loc[np.where(df.field_cd.isnull())[0], 'field_cd'] 

488    1.0
489    1.0
490    1.0
491    1.0
492    1.0
493    1.0
494    1.0
495    1.0
496    1.0
497    1.0
498    1.0
499    1.0
500    1.0
501    1.0
502    1.0
503    1.0
504    NaN
505    NaN
Name: field_cd, dtype: float64

In [42]:
df.loc[488:503, 'field_cd'] = 1.0
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)

In [43]:
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-1)

In [44]:
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

In [45]:
df.loc[np.where(df.career_c.isnull())[0], 'career_c'] 

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
488     1.0
489     1.0
490     1.0
491     1.0
492     1.0
493     1.0
494     1.0
495     1.0
496     1.0
497     1.0
498     1.0
499     1.0
500     1.0
501     1.0
502     1.0
503     1.0
504     NaN
505     NaN
1604    6.0
1605    6.0
1606    6.0
1607    6.0
1608    6.0
1609    6.0
1610    6.0
1611    6.0
1612    6.0
1613    6.0
Name: career_c, dtype: float64

In [46]:
df.loc[488:503, 'career_c'] = 1.0
df.loc[1604:1613, 'career_c'] = 6.0
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)

In [47]:
np.where(df.expnum.isnull())[0].shape

(6434,)

In [48]:
df.loc[1604:1613, 'expnum'] = 3.0
df.loc[:, 'expnum'] = df.loc[:, 'expnum'].fillna(0)

In [49]:
df.loc[np.where(df.date.isnull())[0], 'date'] 

6197    5.0
6198    5.0
6199    5.0
6200    5.0
6201    5.0
6202    5.0
6203    5.0
6204    5.0
6205    5.0
6206    5.0
6207    5.0
6208    5.0
6209    5.0
6210    5.0
6211    5.0
6212    5.0
6213    5.0
6214    5.0
Name: date, dtype: float64

In [50]:
df.loc[6197:6214, 'date'] = 5.0
df.loc[:, 'date'] = df.loc[:, 'date'].fillna(0)
df.loc[6197:6214, 'date'] 

6197    5.0
6198    5.0
6199    5.0
6200    5.0
6201    5.0
6202    5.0
6203    5.0
6204    5.0
6205    5.0
6206    5.0
6207    5.0
6208    5.0
6209    5.0
6210    5.0
6211    5.0
6212    5.0
6213    5.0
6214    5.0
Name: date, dtype: float64

In [51]:
df = df.iloc[:, :18]

In [52]:
df.head()

,iid,gender,pid,match,int_corr,samerace,age,field_cd,race,imprace,imprelig,income,goal,date,go_out,career_c,exphappy,expnum
0,1,0,11.0,0,0.14,0,21.0,1.0,4.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
1,1,0,12.0,0,0.54,0,21.0,1.0,4.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
2,1,0,13.0,1,0.16,1,21.0,1.0,4.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
3,1,0,14.0,1,0.61,0,21.0,1.0,4.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
4,1,0,15.0,1,0.21,0,21.0,1.0,4.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0


In [53]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']).drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

In [54]:
df_pair = df_male.join(df_female.set_index('iid_f'),on='pid',how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)

In [55]:
df_pair = df_pair.drop(['race','pid_f','race_f'], axis=1)
df_pair.head()

,match,int_corr,samerace,age,field_cd,imprace,imprelig,income,goal,date,...,field_cd_f,imprace_f,imprelig_f,income_f,goal_f,date_f,go_out_f,career_c_f,exphappy_f,expnum_f
110,0,0.54,0,22.0,1.0,1.0,1.0,29237.0,1.0,1.0,...,1.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
130,1,0.61,0,23.0,1.0,1.0,1.0,56580.0,2.0,4.0,...,1.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
140,1,0.21,0,24.0,1.0,3.0,1.0,36782.0,1.0,4.0,...,1.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
160,0,0.34,0,30.0,13.0,1.0,1.0,38548.0,3.0,3.0,...,1.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0
170,0,0.50,0,27.0,8.0,8.0,8.0,52010.0,1.0,6.0,...,1.0,2.0,4.0,69487.0,2.0,7.0,1.0,0.0,3.0,2.0


In [56]:
df_pair.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1089 entries, 110 to 8353
Data columns (total 25 columns):
match         1089 non-null int64
int_corr      1089 non-null float64
samerace      1089 non-null int64
age           1089 non-null float64
field_cd      1089 non-null float64
imprace       1089 non-null float64
imprelig      1089 non-null float64
income        1089 non-null float64
goal          1089 non-null float64
date          1089 non-null float64
go_out        1089 non-null float64
career_c      1089 non-null float64
exphappy      1089 non-null float64
expnum        1089 non-null float64
age_f         1089 non-null float64
field_cd_f    1089 non-null float64
imprace_f     1089 non-null float64
imprelig_f    1089 non-null float64
income_f      1089 non-null float64
goal_f        1089 non-null float64
date_f        1089 non-null float64
go_out_f      1089 non-null float64
career_c_f    1089 non-null float64
exphappy_f    1089 non-null float64
expnum_f      1089 non-null flo

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=123)

## Проверка скорости работы на Speed Dating Data 

In [60]:
my_clf = MyDecisionTreeClassifier()
clf = DecisionTreeClassifier()
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [61]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.6499678869621067

In [62]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.47596153846153844

## Проверка качества работы на Speed Dating Data

In [63]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
%time clf.fit(X_train, y_train)

CPU times: user 15.3 ms, sys: 0 ns, total: 15.3 ms
Wall time: 15 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [64]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
%time my_clf.fit(X_train, y_train)

CPU times: user 159 ms, sys: 0 ns, total: 159 ms
Wall time: 158 ms


## Задание 3

In [65]:
features_names = df_pair.columns[1:]
features_names

Index(['int_corr', 'samerace', 'age', 'field_cd', 'imprace', 'imprelig',
       'income', 'goal', 'date', 'go_out', 'career_c', 'exphappy', 'expnum',
       'age_f', 'field_cd_f', 'imprace_f', 'imprelig_f', 'income_f', 'goal_f',
       'date_f', 'go_out_f', 'career_c_f', 'exphappy_f', 'expnum_f'],
      dtype='object')

## Задание 4

In [66]:
print('DecisionTreeClassifier\n')
print(pd.Series(index=features_names, data=clf.feature_importances_).sort_values()[::-1].head(10))

DecisionTreeClassifier

int_corr     0.137282
income_f     0.109776
income       0.067362
imprace_f    0.067063
age_f        0.060328
age          0.058896
imprace      0.051537
goal_f       0.047220
go_out       0.046392
expnum_f     0.045110
dtype: float64


In [67]:
print('MyDecisionTreeClassifier\n')
print(-1 * pd.Series(index=features_names, data=my_clf.feature_importances_).sort_values()[::].head(10))

MyDecisionTreeClassifier

int_corr    0.250000
goal_f      0.083900
career_c    0.035789
expnum_f    0.020809
age_f       0.017608
goal        0.009052
imprelig    0.008273
expnum      0.007972
imprace     0.005503
date_f      0.005159
dtype: float64


## Задание 5

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [69]:
parameters = {
    "criterion": ["gini", "entropy"],
    'max_features': ['sqrt', 'log2', None],
    "max_depth": range(1, 20),
    "min_samples_split": range(2, 20),
}
RandomizedSearchCV(RandomForestClassifier(n_estimators=25), parameters, cv=5).fit(X_train, y_train).best_params_

{'min_samples_split': 4,
 'max_features': 'log2',
 'max_depth': 9,
 'criterion': 'entropy'}